In [1]:
#import lxml.html
#import requests
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import time
import math
import re
import sqlite3
import json
import plotly.graph_objects as go
import plotly.express as px

#from bs4 import BeautifulSoup as bs4
#from selenium.webdriver import Chrome
#from selenium import webdriver
#from selenium.webdriver.common.by import By
from datetime import datetime, timedelta

from sqlalchemy import MetaData, text, Column, Integer, String, ForeignKey, Table, create_engine, Float, Boolean, DateTime
from sqlalchemy.orm import relationship, backref, sessionmaker
from sqlalchemy.ext.declarative import declarative_base

import optimize_lineup as ol
import fantasy_utils as fu
fp = fu.Fantasy_Projections()

In [2]:
conn = sqlite3.connect('fantasy_data.db')
#conn.create_function('sqrt', 1, math.sqrt)
import logging
logging.basicConfig(level=logging.DEBUG)

def safe_sqrt(x):
    try:
        return math.sqrt(x)
    except Exception as e:
        logging.error(f"Error computing sqrt for {x}: {e}")
        return None  # or some other appropriate value or handling

conn.create_function('sqrt', 1, safe_sqrt)

In [3]:
team_list = ['Lima Time', 'Ugly Spuds', 'Harvey', 'Roiders', 'Charmer', 'Wu Tang', '9 Grand Kids', 'Brewbirds', 'Trouble', 'Lil Trump', 'Dirty Birds', 'Young Guns'] 
eligibility_url_dict = {"9 Grand Kids":'30', 'Brewbirds':'3', 'Charmer':'36', 'Dirty Birds':'41', "Harveys Wallbangers":'4', 'Lil Trump & the Ivanabees':'27', 'Lima Time!':'38', 
                        'Roid Ragers':'44', 'Trouble with the Curve':'1', 'Ugly Spuds':'29', 'Wiscompton Wu-Tang':'42', 'Young Guns':'45', "Mom's Cookin":'47'}
stat_list = ['BA', 'HR', 'R', 'RBI', 'SB', 'ERA', 'Sv+Hld', 'SO', 'W', 'WHIP']

opening_week = int(datetime.strftime(pd.Timestamp('2024-03-28'), '%W')) - 1
cur_week = int(datetime.strftime(pd.to_datetime('2024-07-29'), '%W'))
period = cur_week - opening_week
current_period = period + 1
recent_year = pd.read_sql("SELECT max(year) year FROM roster", conn).iloc[0]['year']
recent_week = pd.read_sql(f"SELECT max(week) week FROM roster where year={recent_year}", conn).iloc[0]['week']
opening_week, cur_week, period, current_period, recent_week

(12, 31, 19, 20, 27)

In [4]:
ids = fp.load_id_map()

In [5]:
pd.read_sql("SELECT name FROM sqlite_master", conn)

,name
0,hitting
1,ix_hitting_index
2,hitters
3,sqlite_autoindex_hitters_1
4,players2022
5,owners
6,drafted
7,roster
8,stats
9,players2023


In [48]:
st = pd.read_sql("SELECT p.CBSNAME, o.owner, r.*, s.R, s.RBI, s.HR, s.SB, s.H, s.AB, s.W, s.SO, s.SvHld, s.ER, s.Ha, s.BBa, s.IP, s.outs \
        FROM roster r \
        LEFT JOIN stats s On (r.cbsid=s.cbsid and r.year=s.year and r.week=s.week) \
        LEFT JOIN players p On (p.cbsid=r.cbsid) \
        INNER JOIN owners o On (o.owner_id=r.owner_id) \
        WHERE r.year=2024 AND r.decision='start'", conn).drop_duplicates()
st.shape

(7452, 22)

In [49]:
pd.pivot_table(st, index='owner', columns='week', values='cbsid', aggfunc='count')

week,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
owner,,,,,,,,,,,,,,,,,,,,,,,,,,,
9 Grand Kids,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23
Brewbirds,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23
Charmer,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23
Dirty Birds,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23
Harveys Wallbangers,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23
Lima Time!,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23
Mom's Cookin,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23
Roid Ragers,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23
Trouble with the Curve,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23


In [51]:
st.groupby('owner')[['R','RBI', 'HR', 'SB', 'H', 'AB', 'W', 'SO', 'SvHld', 'ER', 'Ha', 'BBa', 'outs']].sum()

,R,RBI,HR,SB,H,AB,W,SO,SvHld,ER,Ha,BBa,outs
owner,,,,,,,,,,,,,
9 Grand Kids,1001.0,970.0,280.0,167.0,1898.0,7278.0,83.0,1190.0,76.0,520.0,1095.0,355.0,3585.0
Brewbirds,1065.0,1004.0,298.0,183.0,1933.0,7502.0,85.0,1192.0,87.0,462.0,1029.0,333.0,3451.0
Charmer,1039.0,974.0,301.0,248.0,1831.0,7300.0,66.0,1014.0,128.0,397.0,881.0,278.0,3066.0
Dirty Birds,888.0,853.0,224.0,100.0,1687.0,6975.0,73.0,891.0,121.0,416.0,889.0,263.0,2863.0
Harveys Wallbangers,1087.0,965.0,303.0,188.0,1883.0,7422.0,77.0,1106.0,85.0,420.0,901.0,322.0,3236.0
Lima Time!,1016.0,1059.0,292.0,163.0,1934.0,7432.0,67.0,1237.0,87.0,393.0,925.0,323.0,3374.0
Mom's Cookin,942.0,867.0,262.0,128.0,1652.0,6865.0,67.0,1095.0,62.0,438.0,1006.0,363.0,3433.0
Roid Ragers,1023.0,1002.0,304.0,138.0,1864.0,7276.0,67.0,1134.0,55.0,453.0,935.0,354.0,3303.0
Trouble with the Curve,1011.0,958.0,267.0,156.0,1816.0,7028.0,76.0,1125.0,65.0,511.0,1031.0,392.0,3507.0


In [52]:
pd.pivot_table(st[(st['owner']=='Lima Time!') & (~st['pos'].isin(['P', 'SP', 'RP']))], columns='week', values='R', aggfunc='sum').values.cumsum()

array([   5.,   27.,   62.,  118.,  161.,  199.,  239.,  275.,  306.,
        339.,  384.,  421.,  454.,  511.,  550.,  587.,  636.,  651.,
        696.,  739.,  786.,  821.,  854.,  902.,  929.,  965., 1016.])

In [53]:
# Group by owner and sum counting stats
counting_stats = ['R', 'RBI', 'HR', 'SB', 'W', 'SO', 'SvHld', 'H', 'AB', 'Ha', 'BBa', 'ER', 'IP']

# Columns to rank in ascending order (better to have more)
ascending_columns = ['R', 'RBI', 'HR', 'SB', 'W', 'SO', 'SvHld', 'BA']

# Columns to rank in descending order (better to have less)
descending_columns = ['ERA', 'WHIP']

grouped = st.groupby('owner')[counting_stats].sum()

# Calculate rate stats
rate_stats = st.groupby('owner').apply(lambda x: pd.Series({
    'BA': x['H'].sum() / x['AB'].sum(),
    'ERA': x['ER'].sum() / (x['outs'].sum() / 3) * 9,
    'WHIP': (x['Ha'].sum() + x['BBa'].sum()) / (x['outs'].sum() / 3)
}))

standings = pd.concat([grouped, rate_stats], axis=1)

# Rank each column with appropriate ascending argument
ranked_data = {}
for col in ascending_columns:
    ranked_data[col + '_rank'] = standings[col].rank(ascending=True)

for col in descending_columns:
    ranked_data[col + '_rank'] = standings[col].rank(ascending=False)

# Create a DataFrame for ranked data
ranked_df = pd.DataFrame(ranked_data, index=standings.index)
ranked_df['rank'] = ranked_df.sum(axis=1)
ranked_df.sort_values('rank', ascending=False)

,R_rank,RBI_rank,HR_rank,SB_rank,W_rank,SO_rank,SvHld_rank,BA_rank,ERA_rank,WHIP_rank,rank
owner,,,,,,,,,,,
Brewbirds,11.0,11.0,9.0,8.0,12.0,10.0,9.5,9.0,7.0,7.0,93.5
Lima Time!,8.0,12.0,8.0,5.0,4.0,12.0,9.5,11.0,12.0,12.0,93.5
Harveys Wallbangers,12.0,6.0,11.0,10.0,10.0,5.0,8.0,7.0,8.0,10.0,87.0
Charmer,10.0,8.0,10.0,11.0,1.5,2.0,12.0,6.0,9.0,9.0,78.5
9 Grand Kids,5.5,7.0,6.0,7.0,11.0,9.0,7.0,12.0,4.0,4.0,72.5
Roid Ragers,9.0,10.0,12.0,3.0,4.0,7.0,2.0,8.0,6.0,8.0,69.0
Ugly Spuds,2.0,1.0,2.0,12.0,1.5,8.0,6.0,5.0,11.0,11.0,59.5
Trouble with the Curve,7.0,5.0,4.0,4.0,8.5,6.0,5.0,10.0,2.0,3.0,54.5
Wiscompton Wu-Tang,4.0,9.0,5.0,9.0,6.0,11.0,1.0,2.0,1.0,2.0,50.0


In [54]:
def optimize(tm, df):
    df = df.rename(columns={'owner':'Owner', 'CBSNAME':'Player'})
    w = ol.Optimized_Lineups(tm, df[df['Owner']==tm])
    w.catchers = [k for k,v in w.h_dict.items() if 'C,' in v['all_pos']]
    w._make_pitcher_combos()
    print(w.pitcher_optimized_z, w.pitcher_optimized_lineup)
    w._make_hitter_combos()
    print(w.hitter_optimized_z, w.hitter_optimized_lineup)
    return w

In [55]:
def stitch_positions(row):
    position_priority = ['C', '2B', '3B', 'SS', 'OF', '1B', 'MI', 'CI', 'DH', 'SP', 'RP']
    pos_code = row[position_priority+['P']]>=5
    return list(pos_code[pos_code].index)

In [56]:
yr = datetime.now().year
wk = pd.read_sql(f"SELECT max(week) week FROM projections WHERE year={yr}", conn).iloc[0]['week']

print(yr, wk)
proj  = pd.read_sql(f"SELECT distinct p.CBSNAME Player, o.owner Owner, r.pos Decision, j.*, e.*, \
                    CASE WHEN e.DH>=0 AND e.P=0 THEN 'h' ELSE 'p' END As type \
                    FROM roster r \
                    INNER JOIN projections j On (j.cbsid=r.cbsid) \
                    INNER JOIN players p On (r.cbsid=p.cbsid) \
                    INNER JOIN owners o On (r.owner_id=o.owner_id) \
                    INNER JOIN (SELECT cbsid, all_pos, posC C, pos1B '1B', pos2B '2B', pos3B '3B', posSS SS, posOF OF, \
                        posDH DH, posSP SP, posRP RP, posP P FROM eligibility WHERE year={yr} and week={wk}) e \
                        On (r.cbsid=e.cbsid) \
            WHERE j.year={yr} AND j.week={wk} AND j.proj_type='ros' AND r.year=2024 AND r.week={wk-1} \
            ORDER BY Owner, year, week", conn)
proj.rename(columns={'Sv+Hld':'SvHld'}, inplace=True)
proj = proj.loc[:,~proj.columns.duplicated()].copy()
#proj.loc[proj['cbsid']==2942987, 'all_pos'] = 'SS,DH'
#proj.loc[proj['cbsid']==2167479, 'all_pos'] = 'OF,DH'
proj.loc[(proj['1B']>=5) | (proj['3B']>=5), 'CI'] = 5
proj.loc[(proj['2B']>=5) | (proj['SS']>=5), 'MI'] = 5
proj['all_pos'] = proj.apply(lambda x: stitch_positions(x), axis=1).apply(lambda y: ','.join(y))
df2 = proj.copy()
proj[proj['Owner']=='Lima Time!'].shape

2024 28


(32, 39)

In [57]:
recent_week

27

In [60]:
# Optimize lineups based on most recent ROS projections

try:
    optimized_ros = pd.read_csv(f'data/2024-optimized-ros-period-{recent_week+1}.csv').rename(columns={'Unnamed: 0':'Owner'}).set_index('Owner')
except:
    temp = {}

    for tm in standings.index:
        print('\n',tm)
        opt = optimize(tm, proj)
        s = proj[proj['Player'].isin(opt.hitter_optimized_lineup+opt.pitcher_optimized_lineup)][['R', 'RBI', 'HR', 'SB', 'H', 'AB', 'W', 'SO', 'SvHld', 'ER', 'BBa', 'Ha', 'IP']].sum()
        #s['SvHld'] = s['Sv+Hld']
        s['BA'] = s['H']/s['AB']
        s['ERA'] = s['ER']/s['IP']*9
        s['WHIP'] = (s['Ha']+s['BBa'])/s['IP']
        s['hitters'] = opt.hitter_optimized_lineup
        s['hitters_z'] = round(opt.hitter_optimized_z,2)
        s['pitchers'] = opt.pitcher_optimized_lineup
        s['pitchers_z'] = round(opt.pitcher_optimized_z,2)
        s['z'] = round(opt.hitter_optimized_z+opt.pitcher_optimized_z,2)
        temp[tm] = s[['R', 'RBI', 'HR', 'SB', 'W', 'SO', 'SvHld', 'BA', 'ERA', 'WHIP', 'H', 'AB', 'Ha', 'BBa', 'ER', 'IP', 'hitters', 'pitchers', 'hitters_z', 'pitchers_z', 'z']]
    optimized_ros = pd.DataFrame(temp).T
    optimized_ros.to_csv(f'data/2024-optimized-ros-period-{recent_week+1}.csv')


 9 Grand Kids
14.228625617834155 ['George Kirby', 'Carlos Rodon', 'Framber Valdez', 'Kevin Gausman', 'Justin Martinez', 'Seth Lugo', 'Camilo Doval', 'Trevor Megill', 'Alexis Diaz']
15.436618553459475 ['Salvador Perez', 'Connor Wong', 'Ozzie Albies', 'Trea Turner', 'Manny Machado', 'Ezequiel Tovar', 'Ernie Clement', 'Ian Happ', 'Lawrence Butler', 'Lourdes Gurriel', 'Wilyer Abreu', 'Brandon Marsh', 'Carlos Correa', 'Jose Caballero']

 Brewbirds
24.796889221859278 ['Jacob deGrom', 'Chris Sale', 'Logan Webb', 'Clarke Schmidt', 'Ryan Helsley', 'Yu Darvish', 'Cody Bradford', 'Yoshinobu Yamamoto', 'Carlos Estevez']
25.367022552341297 ['Tyler Stephenson', 'Vladimir Guerrero', 'Gleyber Torres', 'Francisco Lindor', 'Jose Ramirez', 'Andres Gimenez', 'Alec Burleson', 'Yordan Alvarez', 'Tommy Edman', 'James Wood', 'Jasson Dominguez', 'Matt Wallner', 'J.D. Martinez', 'Colt Keith']

 Charmer
17.077480360492086 ['Logan Gilbert', 'Corbin Burnes', 'Ryan Walker', 'Cristopher Sanchez', 'Luke Weaver', 'Em

In [61]:
optimized_ros

,R,RBI,HR,SB,W,SO,SvHld,BA,ERA,WHIP,H,AB,Ha,BBa,ER,IP,hitters,pitchers,hitters_z,pitchers_z,z
9 Grand Kids,53.8813,52.4909,14.6396,9.6655,2.6997,46.8036,3.2375,0.259374,3.617696,1.202338,102.8344,396.4721,40.2246,13.2993,17.89413,44.5165,"[Salvador Perez, Connor Wong, Ozzie Albies, Tr...","[George Kirby, Carlos Rodon, Framber Valdez, K...",15.44,14.23,29.67
Brewbirds,56.0583,55.5895,16.13,9.8285,3.1763,53.5611,2.4194,0.263775,3.574201,1.158469,103.6841,393.0776,45.4289,12.5886,19.88892,50.0812,"[Tyler Stephenson, Vladimir Guerrero, Gleyber ...","[Jacob deGrom, Chris Sale, Logan Webb, Clarke ...",25.37,24.8,50.16
Charmer,63.3579,61.7652,19.6785,11.8813,2.8255,46.7198,3.5922,0.260423,3.619579,1.192076,107.5946,413.1531,42.2838,12.5723,18.50703,46.0173,"[Will Smith, Christian Walker, Luis Garcia, Mo...","[Logan Gilbert, Corbin Burnes, Ryan Walker, Cr...",42.43,17.08,59.51
Dirty Birds,53.2294,50.4701,13.0306,8.238,2.6535,44.4719,4.014,0.257099,3.662747,1.204159,101.0418,393.0072,40.2557,12.5003,17.83005,43.8115,"[J.T. Realmuto, Justin Turner, Marcus Semien, ...","[Pablo Lopez, Joe Musgrove, Bailey Ober, Jhoan...",6.4,13.66,20.06
Harveys Wallbangers,51.503,50.454,15.664,9.4873,2.5471,46.6248,3.1889,0.251723,3.759192,1.215758,91.7639,364.5433,37.1473,14.6661,17.80111,42.6182,"[Logan O'Hoppe, Triston Casas, Bryson Stott, G...","[Tarik Skubal, Bryan Woo, Aroldis Chapman, Jef...",8.43,9.08,17.51
Lima Time!,61.2066,59.7757,16.8374,9.9943,2.9479,52.3102,2.9457,0.26511,3.380204,1.155008,113.8198,429.3312,38.6019,14.2909,17.19934,45.7943,"[Yainer Diaz, Josh Naylor, Ketel Marte, C.J. A...","[Garrett Crochet, Cole Ragans, Shota Imanaga, ...",35.05,26.67,61.72
Mom's Cookin,52.9959,51.9773,14.1562,8.4754,2.891,48.3503,2.8996,0.257778,3.814935,1.242444,98.4354,381.8617,44.0472,14.6021,20.00925,47.2048,"[Shea Langeliers, Freddie Freeman, Isiah Kiner...","[Aaron Nola, Max Fried, Nestor Cortes, Kirby Y...",10.23,8.26,18.49
Roid Ragers,56.6556,57.0773,16.9338,8.4527,3.252,55.1472,1.7879,0.256218,3.706686,1.205596,103.7051,404.7528,46.3833,16.3301,21.42407,52.0186,"[Cal Raleigh, Bryce Harper, Luis Arraez, Jerem...","[Zack Wheeler, Gerrit Cole, Mason Miller, Mich...",20.29,17.28,37.57
Trouble with the Curve,57.1307,54.2847,16.1864,9.8171,3.1986,53.999,1.8944,0.256719,3.779084,1.236234,102.3918,398.8481,47.3736,16.4194,21.66788,51.6027,"[Sean Murphy, Yandy Diaz, Joseph Ortiz, Bobby ...","[Dylan Cease, Zach Eflin, Freddy Peralta, Sean...",20.01,11.89,31.9
Ugly Spuds,52.6185,50.5869,14.0268,12.5327,2.5948,47.8148,2.4207,0.248866,3.541243,1.209055,97.8953,393.3652,37.1823,14.5245,16.82732,42.7663,"[Patrick Bailey, Spencer Steer, Tyler Fitzgera...","[Paul Skenes, Blake Snell, Bryce Miller, Seran...",9.2,12.44,21.64


In [62]:
#final_proj = pd.DataFrame([temp[i] for i in list(temp.keys())], index=list(temp.keys())).rename(columns={'Sv+Hld':'SvHld'})+standings
final_proj = optimized_ros+standings
final_proj['BA'] = final_proj['H']/final_proj['AB']
final_proj['ERA'] = final_proj['ER']/final_proj['IP']*9
final_proj['WHIP'] = (final_proj['Ha']+final_proj['BBa'])/final_proj['IP']

In [63]:
final_standings = final_proj[['R', 'RBI', 'HR','SB', 'BA', 'W', 'SO', 'SvHld', 'ERA', 'WHIP']]
for stat in ascending_columns:
    try:
        final_standings[stat+'_rank'] = final_standings.loc[:,stat].rank()
    except:
        print(f'problem with {stat}')
        pass

for stat in descending_columns:
    try:
        final_standings[stat+'_rank'] = final_standings.loc[:,stat].rank(ascending=False)
    except:
        print(f'problem with {stat}')
        pass
final_standings.iloc[:,10:].sum(axis=1).sort_values(ascending=False)
#for i in ['R', 'RBI', 'HR', 'SB', 'BA', 'ERA', 'W', 'SO', 'SvHld', 'WHIP']:
#    final_standings[i+'_rank'] = final_standings[i].rank()

C:\ProgramData\Anaconda3\envs\Python_37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\ProgramData\Anaconda3\envs\Python_37\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Lima Time!                94.0
Brewbirds                 93.0
Harveys Wallbangers       85.0
Charmer                   82.0
9 Grand Kids              72.0
Roid Ragers               70.0
Ugly Spuds                59.0
Trouble with the Curve    55.0
Wiscompton Wu-Tang        49.0
Young Guns                49.0
Mom's Cookin              37.0
Dirty Birds               35.0
dtype: float64

In [22]:
final_standings

,R,RBI,HR,SB,BA,W,SO,SvHld,ERA,WHIP,R_rank,RBI_rank,HR_rank,SB_rank,W_rank,SO_rank,SvHld_rank,BA_rank,ERA_rank,WHIP_rank
9 Grand Kids,1054.8813,1022.4909,294.6396,176.6655,0.260713,85.6997,1236.8036,79.2375,3.905562,1.212982,5.0,7.0,6.0,7.0,11.0,9.0,7.0,12.0,4.0,4.0
Brewbirds,1121.0583,1059.5895,314.13,192.8285,0.257969,88.1763,1245.5611,89.4194,3.612899,1.182933,12.0,11.0,10.0,9.0,12.0,10.0,9.0,9.0,7.0,7.0
Charmer,1102.3579,1035.7652,320.6785,259.8813,0.251336,68.8255,1060.7198,131.5922,3.501473,1.136572,11.0,9.0,11.0,11.0,2.0,2.0,12.0,6.0,9.0,10.0
Dirty Birds,909.2294,875.4701,227.0306,107.238,0.242783,72.6535,896.4719,122.014,3.923363,1.204777,1.0,1.0,1.0,1.0,7.0,1.0,11.0,3.0,3.0,5.0
Harveys Wallbangers,1096.503,983.454,308.664,185.4873,0.253149,76.5471,1108.6248,81.1889,3.538748,1.140254,10.0,4.0,8.0,8.0,8.0,3.0,8.0,7.0,8.0,9.0
Lima Time!,1077.2066,1118.7757,308.8374,172.9943,0.260493,69.9479,1289.3102,89.9457,3.154209,1.111462,8.0,12.0,9.0,5.0,4.0,12.0,10.0,11.0,12.0,12.0
Mom's Cookin,994.9959,918.9773,276.1562,136.4754,0.241544,69.891,1143.3503,64.8996,3.459532,1.19818,3.0,3.0,3.0,2.0,3.0,4.0,3.0,1.0,10.0,6.0
Roid Ragers,1079.6556,1059.0773,320.9338,146.4527,0.256186,70.252,1189.1472,56.7879,3.703099,1.172305,9.0,10.0,12.0,3.0,5.0,7.0,2.0,8.0,6.0,8.0
Trouble with the Curve,1068.1307,1012.2847,283.1864,165.8171,0.258305,79.1986,1178.999,66.8944,3.927448,1.218041,7.0,6.0,4.0,4.0,10.0,6.0,5.0,10.0,2.0,3.0
Ugly Spuds,969.6185,893.5869,239.0268,277.5327,0.245788,68.5948,1224.8148,76.4207,3.358419,1.128497,2.0,2.0,2.0,12.0,1.0,8.0,6.0,5.0,11.0,11.0


In [23]:
29-proj.week.max()

1

In [24]:
stat = 'R'
pt = pd.pivot_table(st, index='owner', columns='week', values=stat, aggfunc='sum')
pt['total'] = pt.sum(axis=1)
pt['mean'] = pt.iloc[:,1:period].mean(axis=1)
pt['std'] = pt.iloc[:,1:period].std(axis=1)
pt['min'] = pt.loc[:,'mean']-(2*pt.loc[:, 'std'])
pt['max'] = pt.loc[:,'mean']+(2*pt.loc[:, 'std'])
v = (pd.pivot_table(proj, index='Owner', values=stat, aggfunc='sum')/(29-proj.week.max())).values
for i in range(22,29):
    pt[i] = v    
pt

week,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,total,mean,std,min,max,28
owner,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9 Grand Kids,0.0,40.0,47.0,45.0,45.0,39.0,30.0,32.0,30.0,44.0,31.0,45.0,43.0,41.0,38.0,47.0,46.0,15.0,45.0,56.0,33.0,64.4852,64.4852,64.4852,64.4852,64.4852,64.4852,1001.0,39.055556,8.467740,22.120076,55.991035,64.4852
Brewbirds,0.0,26.0,41.0,49.0,40.0,37.0,33.0,33.0,42.0,36.0,39.0,47.0,34.0,50.0,42.0,51.0,43.0,16.0,45.0,49.0,49.0,61.7628,61.7628,61.7628,61.7628,61.7628,61.7628,1065.0,39.111111,8.784270,21.542571,56.679651,61.7628
Charmer,8.0,29.0,42.0,39.0,40.0,36.0,31.0,41.0,42.0,51.0,39.0,43.0,49.0,39.0,38.0,42.0,34.0,16.0,38.0,52.0,44.0,71.4816,71.4816,71.4816,71.4816,71.4816,71.4816,1039.0,38.277778,7.714048,22.849682,53.705874,71.4816
Dirty Birds,0.0,24.0,38.0,32.0,40.0,33.0,31.0,29.0,31.0,33.0,31.0,35.0,26.0,43.0,34.0,45.0,30.0,16.0,51.0,29.0,45.0,57.2233,57.2233,57.2233,57.2233,57.2233,57.2233,856.0,33.444444,8.045622,17.353201,49.535688,57.2233
Harveys Wallbangers,4.0,35.0,41.0,61.0,36.0,41.0,39.0,36.0,29.0,43.0,40.0,40.0,48.0,42.0,38.0,41.0,46.0,18.0,37.0,53.0,51.0,51.7688,51.7688,51.7688,51.7688,51.7688,51.7688,1045.0,39.500000,8.500865,22.498270,56.501730,51.7688
Lima Time!,5.0,22.0,35.0,56.0,43.0,38.0,40.0,36.0,31.0,33.0,45.0,37.0,33.0,57.0,39.0,37.0,49.0,15.0,45.0,43.0,47.0,68.8585,68.8585,68.8585,68.8585,68.8585,68.8585,1016.0,38.388889,10.376097,17.636694,59.141084,68.8585
Mom's Cookin,2.0,22.0,30.0,30.0,31.0,36.0,38.0,34.0,47.0,36.0,43.0,35.0,44.0,41.0,47.0,41.0,18.0,21.0,41.0,40.0,46.0,62.6552,62.6552,62.6552,62.6552,62.6552,62.6552,942.0,35.277778,8.648737,17.980304,52.575252,62.6552
Roid Ragers,0.0,25.0,47.0,28.0,41.0,45.0,53.0,34.0,58.0,37.0,31.0,36.0,32.0,40.0,51.0,47.0,30.0,12.0,40.0,40.0,35.0,62.7691,62.7691,62.7691,62.7691,62.7691,62.7691,1023.0,38.166667,11.210552,15.745563,60.587770,62.7691
Trouble with the Curve,0.0,32.0,37.0,50.0,33.0,40.0,34.0,35.0,41.0,43.0,52.0,40.0,38.0,37.0,45.0,40.0,37.0,25.0,42.0,60.0,43.0,69.2357,69.2357,69.2357,69.2357,69.2357,69.2357,1011.0,38.944444,6.384468,26.175507,51.713381,69.2357


In [23]:
optimized_ros.loc['Lima Time!', 'hitters'][2:-2].split("', '")

AttributeError: 'list' object has no attribute 'split'

In [25]:
def chart_stat_ros(st, optimized_ros, tm, stat, recent_week, optimized_alt):
    optimized_ros = optimized_ros.loc[tm]
    pt = pd.pivot_table(st, index='owner', columns='week', values=stat, aggfunc='sum')
    pt['total'] = pt.sum(axis=1)
    pt['mean'] = pt.iloc[:,1:period].mean(axis=1)
    pt['std'] = pt.iloc[:,1:period].std(axis=1)
    pt['min'] = pt.loc[:,'mean']-(2*pt.loc[:, 'std'])
    pt['max'] = pt.loc[:,'mean']+(2*pt.loc[:, 'std'])
    v = (pd.pivot_table(proj, index='Owner', values=stat, aggfunc='sum')/(29-proj.week.max())).values
    for i in range(recent_week,29):
        pt[i] = v
    
    a = optimized_ros.loc[tm]['hitters'][2:-2].split("', '")
    b = optimized_ros.loc[tm,'pitchers'][2:-2].split("', '")
    print(a)
    print(b)
    rbi = proj[proj['Player'].isin(a+b)][['HR', 'SB', 'R', 'RBI', 'W', 'SO', 'SvHld']].sum()[stat]/(29-recent_week)
    rbi2 = df2[df2['Player'].isin(optimized_alt.hitter_optimized_lineup+optimized_alt.pitcher_optimized_lineup)][['HR', 'SB', 'R', 'RBI', 'W', 'SO', 'SvHld']].sum()[stat]/(29-recent_week)

    fig = go.Figure()

    fig.add_trace(
        go.Scatter(name='max',
            x=np.arange(recent_week, 29),
            y=[pt.loc[tm,'total']+(pt.loc[tm, 'max']*i) for i in range(29-recent_week)],
            marker=dict(color='lightgray'),
            #fill='tonexty',
        )
    )
    fig.add_trace(
        go.Scatter(name='min',
            x=np.arange(recent_week, 29),
            y=[pt.loc[tm,'total']+(pt.loc[tm, 'min']*i) for i in range(29-recent_week)],
            marker=dict(color='lightgray'),
            fill='tonexty',
        )
    )
    fig.add_trace(
        go.Scatter(name='actual',
            x=np.arange(1,recent_week+1),
            y=pt.loc[tm,1:recent_week].cumsum().values
        )
    )
    fig.add_trace(
        go.Scatter(name='projection',
            x=np.arange(recent_week,29),
            y=[pt.loc[tm, 'total']]+[(pt.loc[tm, 'total']+(i*rbi)) for i in range(1,29-recent_week)],
            marker=dict(color='gray')
        )
    )
    fig.add_trace(
        go.Scatter(name='trade projection',
            x=np.arange(recent_week,29),
            y=[pt.loc[tm, 'total']]+[(pt.loc[tm, 'total']+(i*rbi2)) for i in range(1,29-recent_week)],
            marker=dict(color='gray'),
            line=dict(dash='dot')
        )
    )
    fig.update_layout(
        title=tm+' '+stat,
        xaxis = dict(dtick=1)
    )
    fig.show()

In [26]:
chart_stat_ros(st, optimized_ros, 'Lima Time!', 'R', recent_week, optimized_alt)

NameError: name 'optimized_alt' is not defined

In [27]:
df2 = proj.copy()

In [28]:
#df2.loc[df2['Player'].isin(['Ryan Mountcastle']), 'Owner'] = "9 Grand Kids"
#df2.loc[df2['Player'].isin(['Jose Caballero']), 'Owner'] = "Lima Time!"
df2.loc[df2['Player'].isin(['Garrett Crochet']), 'Owner'] = "Ugly Spuds"
df2.loc[df2['Player'].isin(['Spencer Steer']), 'Owner'] = "Lima Time!"
#df2.loc[df2['Player'].isin(['Garrett Crochet']), 'Owner'] = 'Ugly Spuds'
#df2.loc[df2['Player'].isin(['Elly De La Cruz']), 'Owner'] = "Lima Time!"

In [29]:
temp_alt = optimized_ros.copy()
optimized_alt = optimize("Ugly Spuds", df2)
temp_alt.loc["Ugly Spuds", ['AB', 'R', 'H', 'HR', 'RBI', 'SB', 'IP', 'W', 'SO', 'SvHld', 'BBa', 'Ha', 'z']] = proj[proj['Player'].isin(optimized_alt.hitter_optimized_lineup+optimized_alt.pitcher_optimized_lineup)][['AB', 'R', 'H', 'HR', 'RBI', 'SB', 'IP', 'W', 'SO', 'SvHld', 'BBa', 'Ha', 'z']].sum()
optimized_alt = optimize('Lima Time!', df2)
temp_alt.loc['Lima Time!', ['AB', 'R', 'H', 'HR', 'RBI', 'SB', 'IP', 'W', 'SO', 'SvHld', 'BBa', 'Ha', 'z']] = proj[proj['Player'].isin(optimized_alt.hitter_optimized_lineup+optimized_alt.pitcher_optimized_lineup)][['AB', 'R', 'H', 'HR', 'RBI', 'SB', 'IP', 'W', 'SO', 'SvHld', 'BBa', 'Ha', 'z']].sum()

#optimized_alt = optimize('Ugly Spuds', df2)
#temp_alt.loc['Ugly Spuds', ['AB', 'R', 'H', 'HR', 'RBI', 'SB', 'IP', 'W', 'SO', 'SvHld', 'BBa', 'Ha', 'z']] = proj[proj['Player'].isin(optimized_alt.hitter_optimized_lineup+optimized_alt.pitcher_optimized_lineup)][['AB', 'R', 'H', 'HR', 'RBI', 'SB', 'IP', 'W', 'SO', 'SvHld', 'BBa', 'Ha', 'z']].sum()

20.21796852745944 ['Paul Skenes', 'Garrett Crochet', 'Blake Snell', 'Bryce Miller', 'Seranthony Dominguez', 'Ryan Pressly', 'Cade Smith', 'Brady Singer', 'Gavin Williams']


IndexError: list index out of range

In [30]:
def see_updated_standings(temp_alt, standings, note):
    final_proj_alt = temp_alt+standings
    final_proj_alt['BA'] = final_proj_alt['H']/final_proj_alt['AB']
    final_proj_alt['ERA'] = final_proj_alt['ER']/final_proj_alt['IP']*9
    final_proj_alt['WHIP'] = (final_proj_alt['Ha']+final_proj_alt['BBa'])/final_proj_alt['IP']

    final_standings_alt = final_proj_alt[['R', 'RBI', 'HR','SB', 'BA', 'W', 'SO', 'SvHld', 'ERA', 'WHIP']]
    for stat in ascending_columns:
        try:
            final_standings_alt[stat+'_rank'] = final_standings_alt.loc[:,stat].rank()
        except:
            print(f'problem with {stat}')
            pass

    for stat in descending_columns:
        try:
            final_standings_alt[stat+'_rank'] = final_standings_alt.loc[:,stat].rank(ascending=False)
        except:
            print(f'problem with {stat}')
            pass
    print(note)
    return final_standings_alt.iloc[:,10:].sum(axis=1).sort_values(ascending=False)

In [31]:
see_updated_standings(temp_alt, standings, "Trade Mountcastle for Caballero")

Trade Mountcastle for Caballero


C:\ProgramData\Anaconda3\envs\Python_37\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\envs\Python_37\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Brewbirds                 96.0
Lima Time!                95.0
Charmer                   83.0
Harveys Wallbangers       73.0
9 Grand Kids              72.0
Roid Ragers               70.0
Ugly Spuds                60.0
Trouble with the Curve    57.0
Young Guns                52.0
Wiscompton Wu-Tang        50.0
Mom's Cookin              38.0
Dirty Birds               34.0
dtype: float64

In [35]:
see_updated_standings(temp_alt, standings, "Trade Reynolds, Alexander for Flaherty, Bibee, Noel")

Trade Reynolds, Alexander for Flaherty, Bibee, Noel


C:\ProgramData\Anaconda3\envs\Python_37\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\envs\Python_37\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Owner
Lima Time!                93.0
Brewbirds                 88.0
Harveys Wallbangers       85.0
9 Grand Kids              82.0
Charmer                   76.0
Ugly Spuds                65.0
Trouble with the Curve    63.0
Wiscompton Wu-Tang        59.0
Roid Ragers               57.0
Young Guns                42.0
Dirty Birds               35.0
Mom's Cookin              35.0
dtype: float64

In [74]:
see_updated_standings(temp_alt, standings, "Trade Kikuchi/Reynolds/Meyer for Lowe/Arozarena/Clase")

Trade Kikuchi/Reynolds/Meyer for Lowe/Arozarena/Clase


C:\ProgramData\Anaconda3\envs\Python_37\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\envs\Python_37\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Owner
Lima Time!                91.0
Brewbirds                 88.0
Charmer                   84.0
9 Grand Kids              82.0
Harveys Wallbangers       82.0
Ugly Spuds                66.0
Trouble with the Curve    62.0
Wiscompton Wu-Tang        59.0
Roid Ragers               58.0
Young Guns                46.0
Dirty Birds               34.0
Mom's Cookin              28.0
dtype: float64

In [61]:
final_standings.iloc[:,10:].sum(axis=1).sort_values(ascending=False)

Lima Time!                92.0
Brewbirds                 88.0
Harveys Wallbangers       83.0
9 Grand Kids              82.0
Charmer                   76.0
Ugly Spuds                65.0
Wiscompton Wu-Tang        63.0
Trouble with the Curve    62.0
Roid Ragers               57.0
Young Guns                42.0
Dirty Birds               35.0
Mom's Cookin              35.0
dtype: float64

In [36]:
proj[proj['Player']=='David Hamilton']

,Player,Owner,Decision,cbsid,year,week,proj_type,AB,R,H,HR,RBI,BB,SB,BA,IP,W,SO,SvHld,BBa,Ha,ERA,WHIP,ER,z,all_pos,C,1B,2B,3B,SS,OF,DH,SP,RP,P,type,CI,MI
176,David Hamilton,Lima Time!,SS,3158363,2024,21,ros,134.353,17.6731,30.6613,3.1192,14.8466,13.6365,12.1771,0.228214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.827885,"2B,SS,MI,DH",0,0,28,0,51,0,5,0,0,0,h,NaN,5.0


In [37]:
chart_stat_ros(st, optimized_ros, 'Lima Time!', 'SB', recent_week, optimized_alt)

TypeError: only list-like objects are allowed to be passed to isin(), you passed a [str]

In [37]:
chosen_stat = 'W'

fig = go.Figure()

for i, row in pd.pivot_table(st[(~st['pos'].isin(['P', 'SP', 'RP']))], index='owner', columns='week', values=chosen_stat, aggfunc='sum').iterrows():
    fig.add_trace(
        go.Scatter(
            name=i,
            x=list(range(1,29)),
            y=row.cumsum(),
            mode = 'markers+lines',
            opacity=.25,
            marker=dict(color='black'),
        )
)
fig.update_layout(
    title=chosen_stat,
    xaxis_dtick=1
)
fig.update_traces(opacity=1, marker=dict(color='blue', line=dict(color='blue')), selector=({'name':'Lima Time!'}))
fig.update_traces(opacity=.75, marker=dict(color='green', line=dict(color='green')), selector=({'name':'Harveys Wallbangers'}))
fig.update_traces(opacity=.75, marker=dict(color='red', line=dict(color='red')), selector=({'name':'Brewbirds'}))
fig.update_traces(opacity=.75, marker=dict(color='orange', line=dict(color='orange')), selector=({'name':'Ugly Spuds'}))
fig.show()

In [40]:
chosen_stat = 'ERA'

fig = go.Figure()

for i, row in pd.pivot_table(st[(st['pos'].isin(['P', 'SP', 'RP']))], index='owner', columns='week', values=chosen_stat, aggfunc='sum').iterrows():
    fig.add_trace(
        go.Scatter(
            name=i,
            x=list(range(1,29)),
            y=row.cumsum(),
            mode = 'markers+lines',
            opacity=.25,
            marker=dict(color='black'),
        )
)
fig.update_layout(
    title=chosen_stat,
    xaxis_dtick=1
)
fig.update_traces(opacity=1, marker=dict(color='blue', line=dict(color='blue')), selector=({'name':'Lima Time!'}))
fig.update_traces(opacity=.75, marker=dict(color='green', line=dict(color='green')), selector=({'name':'Harveys Wallbangers'}))
fig.update_traces(opacity=.75, marker=dict(color='red', line=dict(color='red')), selector=({'name':'Charmer'}))
fig.update_traces(opacity=.75, marker=dict(color='orange', line=dict(color='orange')), selector=({'name':'Ugly Spuds'}))
fig.show()

KeyError: 'ERA'

In [43]:
proj[proj['Owner']=='Lima Time!']

,Player,Owner,Decision,cbsid,year,week,proj_type,AB,R,H,HR,RBI,BB,SB,BA,IP,W,SO,SvHld,BBa,Ha,ERA,WHIP,ER,z,all_pos,C,1B,2B,3B,SS,OF,DH,SP,RP,P,type,CI,MI
161,Yainer Diaz,Lima Time!,C,2829012,2024,22,ros,131.3500,16.9297,36.4212,5.4844,20.2740,6.1622,0.2916,0.277283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.713949,"C,DH",76,0,0,0,0,0,5,0,0,0,h,NaN,NaN
162,Josh Naylor,Lima Time!,1B,2184448,2024,22,ros,151.5920,21.7426,40.8116,7.8362,25.4044,14.0756,2.3150,0.269220,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.793917,"1B,CI,DH",0,100,0,0,0,0,5,0,0,0,h,5.0,NaN
163,Ketel Marte,Lima Time!,2B,1963334,2024,22,ros,163.1700,27.2286,45.8857,7.1204,23.5993,18.4956,2.4145,0.281215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.007398,"2B,MI,DH",0,0,95,0,1,0,5,0,0,0,h,NaN,5.0
164,Alex Bregman,Lima Time!,3B,2184351,2024,22,ros,165.8230,26.1923,43.5710,7.0121,24.0620,20.4338,0.9443,0.262756,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.101563,"3B,CI,DH",0,0,0,113,0,0,5,0,0,0,h,5.0,NaN
165,C.J. Abrams,Lima Time!,SS,3117475,2024,22,ros,168.1380,24.2510,42.7043,5.5292,18.5422,11.0130,10.2990,0.253984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.032872,"SS,MI,DH",0,0,0,0,107,0,5,0,0,0,h,NaN,5.0
166,Nico Hoerner,Lima Time!,MI,2942984,2024,22,ros,156.2320,20.3065,42.7503,2.5863,15.8213,12.1958,7.9641,0.273633,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.095521,"2B,SS,MI,DH",0,0,103,0,10,0,5,0,0,0,h,NaN,5.0
167,Ryan Mountcastle,Lima Time!,CI,2211203,2024,22,ros,136.4330,17.9287,36.0717,5.7546,20.0086,10.4554,1.0272,0.264391,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.539912,"1B,CI,DH",0,100,0,0,0,0,5,0,0,0,h,5.0,NaN
168,TJ Friedl,Lima Time!,OF,2507496,2024,22,ros,149.0800,20.9635,37.1611,5.2946,19.6050,14.8696,5.6150,0.249269,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.573678,"OF,DH",0,0,0,0,0,41,5,0,0,0,h,NaN,NaN
169,David Fry,Lima Time!,OF,2932748,2024,22,ros,101.9600,13.1485,24.4018,3.5349,13.4340,10.8658,1.0735,0.239327,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.481578,"C,OF,1B,CI,DH",23,15,0,4,0,20,5,0,0,0,h,5.0,NaN
170,Brandon Nimmo,Lima Time!,OF,1894663,2024,22,ros,159.3120,24.9409,39.9864,5.5002,19.3095,21.9868,2.4562,0.250994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.209662,"OF,DH",0,0,0,0,0,111,5,0,0,0,h,NaN,NaN


In [41]:
proj[proj['Owner']=='Wiscompton Wu-Tang'].sort_values('z', ascending=False)

,Player,Owner,Decision,cbsid,year,week,proj_type,AB,R,H,HR,RBI,BB,SB,BA,IP,W,SO,SvHld,BBa,Ha,ERA,WHIP,ER,z,all_pos,C,1B,2B,3B,SS,OF,DH,SP,RP,P,type,CI,MI
324,Jose Altuve,Wiscompton Wu-Tang,2B,1839905,2024,21,ros,207.2070,33.4760,56.5170,8.1665,25.1404,19.7914,6.4726,0.272756,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.559498,"2B,MI,DH",0,0,106,0,0,0,5,0,0,0,h,NaN,5.0
331,Luis Robert,Wiscompton Wu-Tang,OF,2826875,2024,21,ros,190.8090,25.2841,47.0368,9.6568,27.8542,12.9927,8.3241,0.246512,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.029651,"OF,DH",0,0,0,0,0,55,5,0,0,0,h,NaN,NaN
340,Tanner Bibee,Wiscompton Wu-Tang,SP,28926327,2024,21,ros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56.6612,3.5225,59.2629,0.0000,15.9839,50.7886,3.74102,1.17845,23.55230,3.059123,"SP,P",0,0,0,0,0,0,0,22,0,5,p,NaN,NaN
325,Alec Bohm,Wiscompton Wu-Tang,3B,2942960,2024,21,ros,195.2640,25.3488,54.4324,5.9649,27.5499,15.4972,1.9940,0.278763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.881080,"3B,1B,CI,DH",0,13,0,97,0,0,5,0,0,0,h,5.0,NaN
342,Jack Flaherty,Wiscompton Wu-Tang,SP,2135266,2024,21,ros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.2098,3.6523,62.4342,0.0000,17.4312,49.3054,3.75820,1.20878,23.05440,2.793544,"SP,P",0,0,0,0,0,0,0,19,0,5,p,NaN,NaN
343,Bailey Ober,Wiscompton Wu-Tang,SP,2837306,2024,21,ros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56.3154,3.4732,58.2655,0.0000,13.7996,52.2844,3.93762,1.17346,24.63880,2.420665,"SP,P",0,0,0,0,0,0,0,21,0,5,p,NaN,NaN
345,Spencer Schwellenbach,Wiscompton Wu-Tang,SP,26911687,2024,21,ros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.4958,3.4134,53.9083,0.1883,13.6311,54.1226,3.90719,1.22088,24.09250,1.530905,"SP,P",0,0,0,0,0,0,0,11,0,5,p,NaN,NaN
335,Jonathan India,Wiscompton Wu-Tang,DH,2942965,2024,21,ros,192.4810,28.6753,48.0982,5.9761,21.6752,23.7531,4.8424,0.249885,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.417917,"2B,MI,DH",0,0,91,0,0,0,12,0,0,0,h,NaN,5.0
327,Willy Adames,Wiscompton Wu-Tang,MI,2165933,2024,21,ros,191.8440,25.7606,45.2775,8.1307,27.7835,22.6570,3.7186,0.236012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.376603,"SS,MI,DH",0,0,0,0,111,0,5,0,0,0,h,NaN,5.0
332,Brent Rooker,Wiscompton Wu-Tang,OF,2837434,2024,21,ros,162.8350,23.4292,38.6835,9.8742,26.7403,18.1857,1.9247,0.237562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.245469,"OF,DH",0,0,0,0,0,14,84,0,0,0,h,NaN,NaN
